<a href="https://colab.research.google.com/github/facundocarballo/ProgramacionConcurrente/blob/main/TP2/Parte2/TP2_Parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programacion Concurrente - TP2 - Parte 2

Para este ejercicio se ha optado por aplicar el tema teórico MPI (Message Passing Interface). La finalidad del ejercicio es ampliar el conocimiento sobras la manera que posee Python para implementar la comunicación entre distintos procesos con el uso de una librería denominada MPI4py [1]. Para más información puede consultar la última revisión en [2].

Este ejercicio fue presentado por alumnos del curso 2020.

## 2.1. Ejercicio - Hola Mundo con MPI

### 2.1.1. Preguntas del ejercicio

Ejecute este ejemplo para
 o más instancias y responda:

a) ¿Qué función realiza la instancia maestra? ¿Qué función realizan las instancias esclavas?

b) ¿Cuántas de esas instancias ejecuta la función main(), initWork() y doWork()?

c) ¿Cómo se diferencian los mensajes de trabajo o de finalización?

d) ¿Cómo implementaría la función BLAS axpy() con este patrón?¿Sería eficiente? Tips: Pide solo el planteo, no la implementación.

e) ¿Qué sucede cuando solo ejecuta con una sola instancia?

f) Punto opcional: El código que ejecutan las instancias esclavas, tienen un error en su lógica. ¿Cómo se podría solucionar?

#### 2.1.2. Armado del ambiente

Es en donde se instalar, por única vez, el módulo MPI4py de Python en el cuaderno.

In [ ]:
! pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365676 sha256=eeb9b2f533791542b63e8b65fb13e65e3ebdead4c827a7d52d58b81a12167172
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


#### 2.1.3. Código del programa en Lenguaje Python

In [ ]:
%%writefile Ejercicio2.py
from mpi4py import MPI
import numpy as np
import time

# --------------------------------------------
# Formulario
Max_tiempo_sleep =   5#@param {type: "slider", min: 1, max: 10}
Min_tiempo_sleep =   0#@param {type: "slider", min: 0, max: 10}
# --------------------------------------------

# --------------------------------------------
# Constantes de comunicacion
WORK_FLAG = 1
END_WORK_FLAG = 2
# --------------------------------------------

def main():
    comm = MPI.COMM_WORLD # Instanciamos el tipo de comunicador a utilizar.
    id = comm.Get_rank()  # Obtenemos el id como atributo del proceso que se ejecuta.

    # Utilizamos el 0 para definir al procesos Maestro, cualquier otro id sera un esclavo.
    if (id == 0) :
        init() # Llamamos funcion init para eventos que requeriremos inicialmente solo 1 vez.
        numProcesses = comm.Get_size()  # Obtenemos el numero de procesos totales ejecutados.
        numTasks = (numProcesses-1)*4 # Se setea el numero de tareas.
        workTimes = generateTasks(numTasks) # Se generan las tareas, en este caso seran
        print("El jefe crea {} horas de descanso de sus empleados:".format(workTimes.size), flush=True)
        print(workTimes, flush=True)
        initWork(comm, workTimes, numProcesses)
    else:
        doWork(comm)

def generateTasks(numTasks):
    #TODO: Cambiar la semilla del random para que se generen efectivamente diferentes numeros.
    np.random.seed(1000)
    return np.random.randint(low=Min_tiempo_sleep, high=Max_tiempo_sleep, size=numTasks)

def init():
  print()
  print("Version MPI4py utilizada: {}".format(MPI.Get_version()), flush=True)
  print()
  print( "------------------------------------", flush=True)
  print( "Sistema de trabajo Suizo:", flush=True)
  print( "------------------------------------", flush=True)
  print()

def initWork(comm, workTimes, numProcesses):
    totalWork = workTimes.size
    workcount = 0
    recvcount = 0

    print("Jefe enviando las tareas iniciales:", flush=True)
    for id in range(1, numProcesses):
        if workcount < totalWork:
            work=workTimes[workcount]
            comm.send(work, dest=id, tag=WORK_FLAG) # Envia mensaje de iniciar trabajo con el dato correspondiente del array.
            workcount += 1
            print("Jefe envia trabajo y {} hs de descanso al empleado {}.".format(work, id), flush=True)
    print( "------------------------------------", flush=True)

    # Mientras haya trabajo, se recibe el resultado de los empleados y se sigue enviando MAS trabajo.
    while (workcount < totalWork) :
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat) # Recivimos resultados de los empleados.
        recvcount += 1
        workerId = stat.Get_source() # Obtenemos el identificador del empleado.
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)
        #send next work
        work=workTimes[workcount]
        comm.send(work, dest=workerId, tag=WORK_FLAG)
        workcount += 1
        print("Jefe envia nuevo trabajo y {} hs de descanso al empleado {}.".format(work, workerId), flush=True)

    while (recvcount < totalWork):
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)

    for id in range(1, numProcesses):
        comm.send(0, dest=id, tag=END_WORK_FLAG)


def doWork(comm):
    while(True):
        stat = MPI.Status() # Obtiene el estado actual del empleado.
        waitTime = comm.recv(source=0, tag=MPI.ANY_TAG, status=stat) # Obtiene lo enviado por el Jefe.
        print("Soy el empleado con id {}, toca descanzo por {} hs.".format(comm.Get_rank(), waitTime), flush=True)

        if (stat.Get_tag() == END_WORK_FLAG):
            print("Marca tarjeta el empleado {}.".format(comm.Get_rank()), flush=True)
            return
        time.sleep(waitTime)
        comm.send(waitTime, dest=0)

main()

Overwriting Ejercicio2.py


#### 2.1.4 Ejecución del programa

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }
NRO =   6#@param {type: "number"}
# --------------------------------------------

! mpirun --allow-run-as-root --oversubscribe -np $NRO python Ejercicio2.py



Version MPI4py utilizada: (3, 1)

------------------------------------
Sistema de trabajo Suizo:
------------------------------------

El jefe crea 20 horas de descanso de sus empleados:
[3 0 3 4 1 0 1 0 1 4 3 4 4 2 2 1 4 4 2 1]
Jefe enviando las tareas iniciales:
Jefe envia trabajo y 3 hs de descanso al empleado 1.
Jefe envia trabajo y 0 hs de descanso al empleado 2.
Jefe envia trabajo y 3 hs de descanso al empleado 3.
Jefe envia trabajo y 4 hs de descanso al empleado 4.
Jefe envia trabajo y 1 hs de descanso al empleado 5.
------------------------------------
Soy el empleado con id 2, toca descanzo por 0 hs.
Jefe recibe trabajo completado 0 del empleado 2.
Jefe envia nuevo trabajo y 0 hs de descanso al empleado 2.
Soy el empleado con id 2, toca descanzo por 0 hs.
Jefe recibe trabajo completado 0 del empleado 2.
Jefe envia nuevo trabajo y 1 hs de descanso al empleado 2.
Soy el empleado con id 2, toca descanzo por 1 hs.
Soy el empleado con id 3, toca descanzo por 3 hs.
Soy el empleado 

## Respuestas Ejercicio 2.1.1

a)
  La función de la instancia maestra es coordinar y supervisar el trabajo de los demás procesos (empleados) en el sistema de trabajo implementado. Coordina la asignación y seguimiento del trabajo, envía tareas a los empleados y recibe los resultados de su trabajo. También se encarga de marcar la tarjeta y finalizar el trabajo cuando se completan todas las tareas.
  Las instancias esclavas esperan recibir tareas del proceso maestro, realizan el trabajo correspondiente (descanso) y envían los resultados al proceso maestro. Además, una instancia esclava finaliza su trabajo y marca la tarjeta cuando recibe un mensaje de finalización con la etiqueta END_WORK_FLAG.

b)
  La función main() se ejecuta en todas las instancias, tanto en la instancia maestra como en las instancias esclavas. La función main() es el punto de entrada del programa y coordina la ejecución general.

  La función initWork() se ejecuta solo en la instancia maestra. se encarga de inicializar el trabajo, enviar las tareas iniciales a los empleados y coordinar el flujo de trabajo.

  La función doWork() se ejecuta solo en las instancias esclavas. Representa el trabajo realizado por cada empleado y se encarga de recibir las tareas del proceso maestro, realizar el trabajo correspondiente y enviar los resultados al proceso maestro.

c)
  Los mensajes de trabajo y de finalización se diferencian mediante el uso de etiquetas en las comunicaciones entre el proceso maestro y los procesos esclavos.

  WORK_FLAG: Esta etiqueta se utiliza para enviar tareas de trabajo a los empleados (procesos esclavos).

  END_WORK_FLAG: Esta etiqueta se utiliza para enviar un mensaje de finalización a los empleados.

d)
  La función BLAS axpy() realiza la operación "a veces x más y" donde "a" es un escalar y "x" e "y" son vectores. Siguiendo el patrón del código proporcionado, se puede implementar la función axpy() distribuyendo el trabajo entre el proceso maestro y los procesos esclavos de la siguiente manera:

  1- El proceso maestro se encargaría de dividir los vectores "x" e "y" en segmentos más pequeños y distribuirlos entre los procesos esclavos. Cada proceso esclavo sería responsable de realizar la operación axpy() en su segmento correspondiente.

  2- Una vez que los procesos esclavos han realizado la operación axpy() en sus segmentos, enviarían los resultados parciales al proceso maestro.

  3- El proceso maestro recolectaría los resultados parciales de los procesos esclavos y los combinaría para obtener el resultado final.

  Esta implementación aprovecharía la capacidad de paralelismo al distribuir la tarea de realizar la operación axpy() entre múltiples procesos esclavos. Sin embargo, la eficiencia de esta implementación dependerá del tamaño de los vectores y la sobrecarga de comunicación entre los procesos.

e)
  Cuando se ejecuta el código con una sola instancia, se ejecuta todo el código en esa instancia como si fuera la instancia maestra y no hay distribución de trabajo a procesos esclavos adicionales. Esto significa que el trabajo se realiza secuencialmente en la instancia maestra y no se aprovecha el paralelismo de múltiples procesos para acelerar el procesamiento.

f) El error de logica esta en el orden de ejecucion de las operaciones **comm.recv()** y **stat.Get_tag()**. **stat** esta inicializado antes de recibir los datos en **comm.recv()** y por ende no se actualizara con la informacion recibida en la ultima operacion de recepcion.

Para solucionar este error, debemos intercambiar el orden de las operaciones. Primero, debemos verificar la etiqueta usando **stat.Get_tag()** y luego recibimos los datos utilizando **comm.recv()**

### Codigo corregido.

```
def doWork(comm):
    while True:
        stat = MPI.Status()  # Obtiene el estado actual del empleado.
        comm.recv(source=0, tag=MPI.ANY_TAG, status=stat)  # Obtiene lo enviado por el Jefe.
        waitTime = stat.Get_tag()

        if waitTime == END_WORK_FLAG:
            print("Marca tarjeta el empleado {}.".format(comm.Get_rank()), flush=True)
            return

        print("Soy el empleado con id {}, toca descanso por {} hs.".format(comm.Get_rank(), waitTime), flush=True)
        time.sleep(waitTime)
        comm.send(waitTime, dest=0)

```

